In [ ]:
%reload_ext autoreload
%autoreload 2
import zeus.notebook_utils.syspath as syspath
syspath.add_parent_folder()

In [ ]:
from multiprocessing import cpu_count
from zeus.utils import home
from kidney.datasets.toy import create_data_loaders
from kidney.experiments.toy import ToyExperiment
from kidney.utils.checkpoints import CheckpointsStorage

In [ ]:
toy, checkpoint = CheckpointsStorage.create(
    ToyExperiment, home("experiments/toy/checkpoints"), "avg_val_loss"
)

In [ ]:
info = checkpoint.meta

In [ ]:
info.keys()

In [ ]:
loaders = create_data_loaders(info['data'], info['transformers'], num_workers=cpu_count())

In [ ]:
batch = next(iter(loaders['valid']))

In [ ]:
outputs = toy(batch)['outputs']

In [ ]:
outputs.shape